In [1]:
import requests
import bs4
import os
import re
import unicodedata
import pandas as pd
import nltk
import numpy as np

In [2]:
url = "https://www.ebay.com/sch/i.html?_from=R40&_nkw=rtx+3060&_sacat=175673&rt=nc&Chipset%252FGPU%2520Model=NVIDIA%2520GeForce%2520RTX%25203060%2520Ti%7CNVIDIA%2520GeForce%2520RTX%25203060&_dcat=27386&_ipg=200"
response = requests.get(url)
html = response.text
soup = bs4.BeautifulSoup(html)

In [3]:
response

<Response [200]>

In [4]:
items = soup.find_all('div', {'class': 's-item__info clearfix'})

len(items)

211

In [5]:
items[8].find('h3', {'class': "s-item__title"}).text

'New ListingEVGA GeForce RTX 3060 XC GAMING 12GB GDDR6 Graphics Card (12G-P5-3657-KR)'

In [6]:
items[8].find('span', {'class': 'SECONDARY_INFO'}).text

'Brand New'

In [7]:
items[8].find('span', {'class': 's-item__price'}).text

'$640.00'

In [8]:
items[8].find('span', {'class': 's-item__bids s-item__bidCount'}).text

'17 bids'

In [9]:
items[8].find('span', {'class': 's-item__time-left',}).text

'10h 32m left'

In [10]:
items[8].find('span', {'class': 's-item__shipping s-item__logisticsCost'}).text

'+$14.20 shipping'

In [21]:
rows = []

for item in items:
    
    title = item.find('h3', {'class': "s-item__title"}).text
    condition = item.find('span', {'class': 'SECONDARY_INFO'})
    current_price = item.find('span', {'class': 's-item__price'})
    total_bids = item.find('span', {'class': 's-item__bids s-item__bidCount'})
    bidding_time_left = item.find('span', {'class': 's-item__time-left',})
    shipping_cost = item.find('span', {'class': 's-item__shipping s-item__logisticsCost'})
    
    if condition is None:
        condition = np.nan
    else:
        condition = condition.text
        
    if current_price is None:
        current_price = np.nan
    else:
        current_price = current_price.text

    if total_bids is None:
        total_bids = np.nan
    else:
        total_bids = total_bids.text
        
    if bidding_time_left is None:
        bidding_time_left = np.nan
    else:
        bidding_time_left = bidding_time_left.text
        
    if shipping_cost is None:
        shipping_cost = np.nan
    else:
        shipping_cost = shipping_cost.text
        
    row = []
    row.append(title)
    row.append(condition)
    row.append(current_price)
    row.append(total_bids)
    row.append(bidding_time_left)
    row.append(shipping_cost)
    rows.append(row)
    
df = pd.DataFrame(columns=['title', 'condition', 'current_price', 'total_bids', 'bidding_time_left', 'shipping_cost'], data = rows)

df


,title,condition,current_price,total_bids,bidding_time_left,shipping_cost
0,,NaN,NaN,NaN,NaN,NaN
1,EVGA GeForce RTX 3060 XC GAMING 12GB GDDR6 Gra...,Brand New,$710.00,5 bids,5m left,+$17.20 shipping
2,EVGA GeForce RTX 3060 XC GAMING 12GB GDDR6 Gra...,Brand New,$689.99,NaN,NaN,+$13.00 shipping
3,NEW EVGA GeForce RTX 3060 XC GAMING 12GB GDDR6...,Brand New,$719.99,NaN,NaN,NaN
4,ASUS DUAL GeForce RTX 3060 OC Edition 12GB GDD...,Brand New,$665.00,NaN,NaN,+$13.50 shipping
...,...,...,...,...,...,...
206,ZOTAC Gaming GeForce RTX 3060 Twin Edge OC 12G...,Brand New,$695.99,NaN,NaN,Free shipping
207,EVGA GeForce RTX 3060 XC GAMING 12GB GDDR6 Gra...,Brand New,$740.00,NaN,NaN,Free shipping
208,EVGA GeForce RTX 3060 XC3 Gaming 12GB GDDR6 Gr...,Brand New,$699.00,NaN,NaN,+$24.99 shipping
209,EVGA GeForce RTX 3060 XC GAMING 12GB Graphics ...,Brand New,$739.99,NaN,NaN,Free shipping


In [68]:
regex = r'(\d*)\,(\d+)'
subject = soup.find('h1', {'class': 'srp-controls__count-heading'}).text

str_num = re.findall(regex, subject)

In [75]:
str_num


[('1', '265')]